In [19]:
import os
import cv2
import sys
from pathlib import Path

In [ ]:
# Size restrictions 
new_size = (128, 128)

# Image Directories
input_directory = Path('data/raw')
output_directory = Path('data/processed')

# Creating the output directory incase the directory does not exists
os.makedirs(output_directory, exist_ok=True)
# Checking if the input directory exists 

if not os.path.isdir(input_directory):
    print(f"Directory '{input_directory}' not found.")
    sys.exit(1)

# Function to resize the image 
def paddedResize(image):
    # # Storing the new width and new height in a variable
    new_width, new_height = new_size
    
    # # Storing the old width and old height in a variable 
    old_height, old_width = image.shape[:2]

    # Determining the scale in the image
    scale = 128 / max(old_height, old_width)

    # Determining the new width and new height by the scale 
    new_height = int(old_height * scale)
    new_width = int(old_width * scale)

    # # Computing the difference in height and width
    # height_allignment = old_height - new_height
    # width_allignment = old_width - new_width

    # # Compute the aspect ratio for not overstreaching the images
    # cropped_image_top = height_allignment // 2
    # cropped_image_left = width_allignment // 2

    # Customize the dimensions for better storage and transformation
    image_top = (128 - new_height) // 2
    image_left = (128 - new_width) // 2
    image_right = (128 - image_left - new_width)
    image_bottom = (128 - image_top - new_height)

    # Resizing the image to the given height and width and downstream the area of the image without casuing any pixel distortion [INTER_AREA]
    resize_image = cv2.resize(
        image, 
        (new_width, new_height), 
        interpolation=cv2.INTER_AREA)

    # # Add Padding to all the size to prevent image and feature distortion
    image_padding = cv2.copyMakeBorder(
        resize_image,
        image_top,
        image_bottom,
        image_left,
        image_right,
        cv2.BORDER_CONSTANT,
    )

    return image_padding

# For loop to transverse/walking through all the images in folders present in the directory 
for current_address, directory, file_name in os.walk(input_directory):
    for individual_files in file_name:
        # Return true if the image file is a jpeg/jpeg or png file 
        if not individual_files.lower().endswith(('jpg', 'jpeg', 'png')):
            continue
        
        # Storing the image path in a variable
        image_individual_path = os.path.join(current_address, individual_files)
        # Storing the raw image file in an Open CV object
        raw_image = cv2.imread(image_individual_path)

        # Converting the raw image to grey
        raw_image = cv2.cvtColor(raw_image, cv2.COLOR_BGR2GRAY)

        # Resize the image with padding 
        processed_image = paddedResize(raw_image)

        # Storing the object into a specific output folder
        file_output_path = Path(output_directory) / Path(current_address).relative_to(input_directory) / individual_files
        # If the path does not exists for the directory, make one
        if not file_output_path.exists():
            file_output_path.parent.mkdir(parents=True, exist_ok=True)

        # Save the image in the directory
        cv2.imwrite(str(file_output_path), processed_image)

        print("File successfully processed:", image_individual_path)